# Supervised Trading on Key Sentiment

#### STonKS

In [2]:
import pymongo
from pymongo import MongoClient
import pandas as pd
import praw
import time
import datetime
from datetime import datetime, timedelta
import re

In [3]:
client = MongoClient('localhost', 27017)
db = client.db
headlines = db.headlines
threads = db.threads
analysis = db.analysis

In [4]:
df = pd.read_csv('companylist.csv')

In [5]:
secretfile = open('secret.txt')
reddit = praw.Reddit(client_id='-wpcPIbA7bhlpw', client_secret=secretfile.read(), user_agent='sentiment')

In [6]:
wantedTags = ['Daily Discussion', 'DD', 'Technicals', 'Fundamentals', 'Discussion', 'YOLO', 'Stocks']
subreddit = reddit.subreddit('wallstreetbets')
postlist = []

for post in subreddit.hot(limit = 1000):
    if datetime.today().weekday == 0:
        if len(postlist) < 100:
            postTime = post.created
            postDate = datetime.utcfromtimestamp(postTime)
            
            currentTime = datetime.utcnow()
            
            postDelta = currentTime - postDate

            if postDelta.days < 3 and post.link_flair_text in wantedTags:
                postlist.append(post)
        else:
            break
    
    else:
        if len(postlist) < 100:
            postTime = post.created
            postDate = datetime.utcfromtimestamp(postTime)

            currentTime = datetime.utcnow()

            postDelta = currentTime - postDate
            postDelta = str(postDelta)

            if 'day' not in postDelta and post.link_flair_text in wantedTags:
                postlist.append(post)
        else:
            break

In [7]:
postDf = pd.DataFrame(columns = ['Date','Title','Body'])

In [8]:
dates = '{}/{}/{}'.format(currentTime.month,currentTime.day,currentTime.year)
titles = [post.title for post in postlist]
bodies = [post.selftext for post in postlist]
tags = [post.link_flair_text for post in postlist]

In [9]:
postDf['Title'] = titles
postDf['Body'] = bodies
postDf['Date'] = dates
postDf['Tag'] = tags

In [10]:
def cleanStrings(string):
    return re.sub('[^a-zA-Z0-9./$]+', ' ',string)

In [11]:
postDf['Title'] = postDf['Title'].apply(cleanStrings)
postDf['Body'] = postDf['Body'].apply(cleanStrings)

In [12]:
stocklist = df['Symbol']

In [13]:
stocklist = [s for s in stocklist if s.isalpha()]
stocklistcopy = ['$'+ s for s in stocklist]
stocklist = [s for s in stocklist if len(s) >= 3]
stocklist = stocklist + stocklistcopy

In [14]:
def findStocks(string):
    stockset = set(string.split()).intersection(stocklist)
    return stockset

In [15]:
postDf['set1'] = postDf['Body'].apply(findStocks)
postDf['set2'] = postDf['Title'].apply(findStocks)
postDf['Stocks'] = postDf.apply(lambda x: x['set1'].union(x['set2']), axis = 1)
postDf = postDf.drop(columns = ['set1','set2'])

In [16]:
postDf

,Date,Title,Body,Tag,Stocks
0,9/28/2020,Fine. Fuck my losses. I hope one of us made co...,Seriously. From March to now at least fucking ...,Discussion,{}
1,9/28/2020,Take advantage of the EV goldrush by completin...,As you may already know we are currently in th...,Discussion,"{TSLA, HCA}"
2,9/28/2020,Mining to the moon,I ve been sucking Peter Schiff s golden dick f...,Discussion,{NOW}
3,9/28/2020,FT Nikola founder bought truck designs from th...,https //www.ft.com/content/9bd5e3f3 4d09 458c...,Discussion,{}
4,9/28/2020,Nikola is completely Fucked. Trevor Milton and...,TL DR buy puts I hope you Retards are ready fo...,Discussion,{NKLA}
5,9/28/2020,This is The Bottom,Bears feasting on apples post getting the mos...,Discussion,"{EDIT, QQQ, $TSLA, $NVTA, $FB}"
6,9/28/2020,Silver is looking bullish on a larger time scale,,DD,{}
7,9/28/2020,Is the average WSB redditor more affluent than...,I hypothesize that most people on this subredd...,Discussion,{}
8,9/28/2020,Public Service Announcement For Every Retail T...,If you are a retail trader. The big boys and t...,Discussion,{}
9,9/28/2020,Where is the gecko,Everyone remembers that bug eating gecko a cou...,Discussion,{}


In [17]:
df['Name'].unique().size

4790

In [18]:
df

,Symbol,Name,LastSale,MarketCap,Country,IPOyear,Sector,Industry
0,TXG,"10x Genomics, Inc.",121.8900,1.272031e+10,United States,2019.0,Capital Goods,Biotechnology: Laboratory Analytical Instruments
1,PIH,"1347 Property Insurance Holdings, Inc.",3.9700,2.409038e+07,United States,2014.0,Finance,Property-Casualty Insurers
2,PIHPP,"1347 Property Insurance Holdings, Inc.",24.9999,0.000000e+00,United States,NaN,Finance,Property-Casualty Insurers
3,TURN,180 Degree Capital Corp.,1.9702,6.131570e+07,United States,NaN,Finance,Finance/Investors Services
4,FLWS,"1-800-FLOWERS.COM, Inc.",23.0400,1.484037e+09,United States,1999.0,Consumer Services,Other Specialty Stores
...,...,...,...,...,...,...,...,...
5716,ZUMZ,Zumiez Inc.,30.0100,7.636060e+08,United States,2005.0,Consumer Services,Clothing/Shoe/Accessory Stores
5717,ZYME,Zymeworks Inc.,45.3400,2.068740e+09,Canada,2017.0,Health Care,Major Pharmaceuticals
5718,ZYNE,"Zynerba Pharmaceuticals, Inc.",3.7000,1.082441e+08,United States,2015.0,Health Care,Major Pharmaceuticals
5719,ZYXI,"Zynex, Inc.",17.2000,5.969356e+08,United States,NaN,Health Care,Biotechnology: Electromedical & Electrotherape...


In [19]:
test = df.sort_values(by = ['MarketCap'], ascending = False).reset_index()

In [20]:
test.loc[test['Symbol'] == 'COST']

,index,Symbol,Name,LastSale,MarketCap,Country,IPOyear,Sector,Industry
39,1322,COST,Costco Wholesale Corporation,335.96,1.483340e+11,United States,NaN,Consumer Services,Department/Specialty Retail Stores


In [21]:
unwantedEndings = ['Inc.', 'Corp.', 'Corporation', 'Company', 'Inc', 'L.P.', 'LLC']

In [22]:
def cleanName(string):
    for word in unwantedEndings:
        string = string.replace(word,'')
    string = re.sub('[^a-zA-Z0-9.]+', ' ',string)
    return string

In [26]:
companiesList = ', '.join(set(df['Name'].apply(cleanName)))

In [27]:
companiesList = companiesList.lower().split()
companiesList2 = []
nameDict  = {}

for i in companiesList:
    if i not in companiesList2:
        companiesList2.append(i)
        
for i in range(0,len(companiesList2)):
    nameDict.update({companiesList2[i]:companiesList.count(companiesList2[i])})

In [30]:
nameDict['american']

39